In [1]:
#перевод RGB в HSV

def rgb_to_hsv(px):
    r = px[0] / 255
    g = px[1] / 255
    b = px[2] / 255
    
    mx = max(r, g, b)
    mn = min(r, g, b)
    
    h, s, v = 0, 0, mx
    if mx != mn and mx == r and g >= b:
        h = 60 * (g - b) / (mx - mn)
    elif mx != mn and mx == r and g < b:
        h = 60 * (g - b) / (mx - mn) + 360
    elif mx != mn and mx == g:
        h = 60 * (b - r) / (mx - mn) + 120
    elif mx != mn and mx == b:
        h = 60 * (r - g) / (mx - mn) + 240
    
    if mx != 0:
        s = 1 - mn / mx

    return (h, s, v)

#нормировка HSV в RGB для удобства пользования изображениями

def converted_hsv(px):
    h, s, v = rgb_to_hsv(px)

    r = int(h / 360 * 255)
    g = int(s * 255)
    b = int(v * 255)
    return (r, g, b)

#перевод картинки в HSV

def image_to_hsv(img):
    w = img.width
    h = img.height
    img2 = img.copy()
    for i in range(w):
        for j in range(h):
            img2.putpixel((i, j), converted_hsv(img.getpixel((i, j))))
    return img2



def green_based_rgb(px, coef, dist, light):
    h, s, v = rgb_to_hsv(px)
    if 120 - dist <= h and h <= 120 + dist and s > light:
        return (px[0], px[1], px[2])
    return (int(px[0] // coef), int(px[1] // coef), int(px[2] // coef))

def to_green_based_hsv(img, coef, dist, light):
    w = img.width
    h = img.height
    img2 = img.copy()
    for i in range(w):
        for j in range(h):
            img2.putpixel((i, j), green_based_rgb(img.getpixel((i, j)), coef, dist, light))
    return img2

In [35]:
from PIL import Image
from rembg.bg import remove

img = Image.open("sample/IMG_4703.jpg")
alt = remove(img)
hsv_alt = remove(to_green_based_hsv(img, 10, 40, 0.1))

In [40]:
import os
dataFolder = "Data400"
out = "MonoRemBg"
for cls in os.listdir(dataFolder):
    for img in os.listdir(dataFolder+"/"+cls):
        erasedBack = remove(Image.open(dataFolder+"/"+cls+"/"+img))
        erasedBack.convert('RGB').save(out+"/"+cls+"/"+img)